In [ ]:
import pandas as pd
import numpy as np
import math as m
import os

### Velocity Features

* mean velocity
* velocity standard deviation
* maximum velocity
* normal velocity variability - Smoother movements incur a lower NVV compared to more irregular movements
* number of peak velocity points (local maxima): The ideal number of maxima for a single upstroke is 1; 
  the greater the number of extrema, the more dysfluent the writing. 

In [ ]:
# mean velocity
def velocity_mean(csv_file): 
    df = pd.read_csv(csv_file)
    return np.mean(np.array(df['Velocity']))

# velocity standard deviation
def velocity_std(csv_file): 
    df = pd.read_csv(csv_file)
    return np.std(np.array(df['Velocity']))

# maximum velocity
def velocity_max(csv_file): 
    df = pd.read_csv(csv_file)
    return np.max(np.array(df['Velocity']))

# normal velocity variability
def velocity_nvv(csv_file): 
    df = pd.read_csv(csv_file)
    sigma_sum = 0
    for i in range(1, len(df['Velocity']) - 1): 
        sigma_sum += abs(df['Velocity'][i+1] - df['Velocity'][i])
    T = duration_calc(csv_file)
    nvv = 1/(T * abs(velocity_mean(csv_file))) * sigma_sum
    return nvv

# number of peak velocity points (number of inversions in velocity - NIV)
from scipy.signal import argrelextrema
def velocity_niv(csv_file): 
    df = pd.read_csv(csv_file)
    velocity = []

    for i in df['Velocity']: 
        velocity.append(i) 

    maximum = argrelextrema(np.array(velocity), np.greater)    
    return len(maximum[0])

### Acceleration Features 

* mean acceleration
* acceleration standard deviation
* maximum acceleration
* number of inversions in acceleration (NIA): number of peak acceleration points (local maxima) - the ideal 
  number of maxima for a single upstroke is 1; the greater the number of extrema, the more dysfluent the 
  writing. 

In [ ]:
# calculating acceleration at each individual point in the drawing 
def accel_calc(csv_file): 
    df = pd.read_csv(csv_file)
    accel = []
    for i in range(2, len(df['Velocity'])): 
        accel.append(abs((df['Velocity'][i]-df['Velocity'][i-1])/(df['Timestamp'][i]-df['Timestamp'][i-1])))
    
    return np.array(accel)

# mean acceleration
def accel_mean(csv_file): 
    accel = accel_calc(csv_file)
    return np.mean(accel)

# acceleration standard deviation
def accel_std(csv_file): 
    accel = accel_calc(csv_file)
    return np.std(accel)

# maximum acceleration
def accel_max(csv_file): 
    accel = accel_calc(csv_file)
    return np.max(accel)

# number of peak acceleration points (number of inversions in acceleration - NIA)
from scipy.signal import argrelextrema
def accel_nia(csv_file): 
    acceleration = accel_calc(csv_file)
    maximum = argrelextrema(acceleration, np.greater)    

    return len(maximum[0])

### Jerk Features

* mean jerk 
* jerk standard deviation
* maximum jerk 
* number of inversions in jerk (NIJ)

In [ ]:
# calculates jerk for each individual point on the drawing 
def jerk_calc(csv_file): 
    df = pd.read_csv(csv_file)
    accel = accel_calc(csv_file)
    jerk = []

    for i in range(50, accel.size): 
        jerk.append(abs((accel[i]-accel[i-1])/(df['Timestamp'][i]-df['Timestamp'][i-1])))
    
    return np.array(jerk)

# mean jerk 
def jerk_mean(csv_file): 
    jerk = jerk_calc(csv_file)
    return np.mean(jerk)

# jerk standard deviation
def jerk_std(csv_file): 
    jerk = jerk_calc(csv_file)
    return np.std(jerk)

# maximum jerk
def jerk_max(csv_file): 
    jerk = jerk_calc(csv_file)
    return np.max(jerk)

# number of inversions in jerk (NIJ) (local maxima)
from scipy.signal import argrelextrema
def jerk_nij(csv_file): 
    jerk = jerk_calc(csv_file)
    maximum = argrelextrema(jerk, np.greater)    

    return len(maximum[0])

### Duration of Drawing Feature 

In [ ]:
def duration_calc(csv_file): 
    df = pd.read_csv(csv_file)
    # len(df['Timetsamp']) - 1 bc there is an empty row at the end of in-house csv's; not sure if this is 
    # the case for PaHaW datasets
    total_duration = df['Timestamp'][len(df['Timestamp']) - 1] - df['Timestamp'][0] 
    return total_duration

### Total Displacement of Drawing Feature

In [ ]:
def total_displacement_calc(csv_file): 
    df = pd.read_csv(csv_file)
    
    displacement = []
    for i in range(len(df['Coordinates']) - 1): 
        x1 = float(df['Coordinates'][i].split(',')[0])
        x2 = float(df['Coordinates'][i+1].split(',')[0])
        y1 = float(df['Coordinates'][i].split(',')[1])
        y2 = float(df['Coordinates'][i+1].split(',')[1])

        displacement.append(np.sqrt((x2-x1)**2 + (y2-y1)**2))
    
    total_displacement = np.sum(np.array(displacement))
    return total_displacement

### Curvature Features

* curvature standard deviation
* absolute difference between mean curvature of the drawing and the constant curvature of the circle being
traced over


In [ ]:
# calculates curvature for each individual point on the drawing (returns a numpy array);
# also prints the number of NaN values so we know if it makes sense to just drop all the nan values in 
# further calculations
def curvature_calc(csv_file, toPrint = False): # where csv_file is a file pathway
    dataset = pd.read_csv(csv_file)

    # reshaping data for np.gradient
    lst = []
    for i in dataset['Coordinates']: 
        lst.append([float(i.split(',')[0]), float(i.split(',')[1])])
    
    # finding dx_dt, dy_dt, d2x_dt2, d2y_dt2
    coordinates = np.array(lst)
    dx_dt = np.gradient(coordinates[20:, 0])
    dy_dt = np.gradient(coordinates[20:, 1])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)

    # calculate curvature using curvature formula for 2D plane curves
    curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5

    if toPrint == True: 
        nan_values = []
        for i in curvature: 
            if m.isnan(i): 
                nan_values.append(i)
        print('Number of NaN values: {}'.format(len(nan_values)))
    
    return curvature

# curvature standard deviation
def curvature_std(csv_file): 
    curvature = curvature_calc(csv_file)
    return np.nanstd(curvature)

# absolute difference between mean curvature of drawing and constant curvature of circle being traced over
def curvature_diff(drawing_csv, circle_csv): # set a default value (path) to the trace circle csv parameter when data is available
    drawing_curv = curvature_calc(drawing_csv)
    circle_curv = curvature_calc(circle_csv)

    # circle_curv should contain a list of the same values, so we can just take the value at the first index
    abs_diff = abs(np.nanmean(drawing_curv) - circle_curv[0]) 
    return abs_diff


### Force Features

* mean force
* force standard deviation
* maximum force

In [ ]:
# mean force
def force_mean(csv_file): 
    df = pd.read_csv(csv_file)
    return np.mean(np.array(df['force']))

# force standard deviation
def force_std(csv_file): 
    df = pd.read_csv(csv_file)
    return np.std(np.array(df['force']))

# maximum force
def force_max(csv_file): 
    df = pd.read_csv(csv_file)
    return np.max(np.array(df['force']))

### Error
* mean error between drawing and the circle being traced over 

In [ ]:
# mean error 
def error_mean(drawing_csv, trace_csv): 
    drawing_df = pd.read_csv(drawing_csv)
    trace_df = pd.read_csv(trace_csv)

    drawing_coors = []
    for i in drawing_df['Coordinates']: 
        drawing_coors.append([float(i.split(',')[0]), float(i.split(',')[1])])

    trace_coors = []
    for i in trace_df['Coordinates']: 
        trace_coors.append([float(i.split(',')[0]), float(i.split(',')[1])])

    def displacement(p1, p2): 
        return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

    def closest(a, B): 
        min_disp = displacement(a, B[0])
        for b in B: 
            if displacement(a, b) < min_disp: 
                min_disp = displacement(a, b)
        return min_disp
    
    error = 0
    for i in drawing_coors: 
        error += closest(i, trace_coors)

    return error/len(drawing_coors) 

### Number of Strokes Feature

While this number is likely to be 1 for the circle drawings, there is a possibility that patients might 
finish a circle with more than 1 stroke. We can implement a number of stroke feature based on segments in 
between points where the force equals 0. This feature will also be useful if we have handwriting tests. 

In [ ]:
def numStroke_calc(csv_file): 
    df = pd.read_csv(csv_file)
    last_index = len(df.index)
    num_strokes = 0

    # the zero_indices list will contain the indices at which force = 0, including -1 and last_index to count stroke #
    zero_indices = [-1]
    for i in range(len(df['force'])): 
        if df['force'][i] == 0: 
            zero_indices.append(i)
    zero_indices.append(last_index)
    
    for i in range(1, len(zero_indices)):
        if zero_indices[i] != (zero_indices[i-1]+1):
            num_strokes +=1
    
    return num_strokes

In [ ]:
# constructs the Pandas DataFrame for gathering all the features
# appends the extracted features from all the drawing csv's from one patient to the frame_list
# requires a frame list 
def add_drawing_csv(directory_path, frame_list): # where the directory contains all the drawings of a patient
    drawing_labels = []
    mean_velocities = []
    std_velocities = []
    max_velocities = []
    nvv = []
    niv = []
    mean_accels = []
    std_accels = []
    max_accels = []
    nia = []
    mean_jerk = []
    std_jerk = []
    max_jerk = []
    nij = []
    std_curvature = []
    absdiff_curvature = [] # need to finish!
    duration = []
    total_displacement = []
    error = [] # need to finish this!
    num_of_strokes = [] # need to finish!
    mean_force = []
    std_force = []
    max_force = []

    directory = os.fsencode(directory_path)
    for file in os.listdir(directory): 
        filename = os.fsdecode(file)
        path = os.path.join(directory_path, filename)

        drawing_labels.append(filename)
        mean_velocities.append(velocity_mean(path))
        std_velocities.append(velocity_std(path))
        max_velocities.append(velocity_max(path))
        nvv.append(velocity_nvv(path))
        niv.append(velocity_niv(path))
        mean_accels.append(accel_mean(path))
        std_accels.append(accel_std(path))
        max_accels.append(accel_max(path))
        nia.append(accel_nia(path))
        
        mean_jerk.append(jerk_mean(path))
        std_jerk.append(jerk_std(path))
        max_jerk.append(jerk_max(path))
        nij.append(jerk_nij(path))
        std_curvature.append(curvature_std(path))
        # absdiff_curvature.append(curvature_diff(path))
        duration.append(duration_calc(path))
        total_displacement.append(total_displacement_calc(path))
        # error.append()
        num_of_strokes.append(numStroke_calc(path))
        mean_force.append(force_mean(path))
        std_force.append(force_std(path))
        max_force.append(force_max(path))

    df = pd.DataFrame({
        'Drawing Label': drawing_labels, 
        'Mean Velocity': mean_velocities, 'Velocity Std': std_velocities, 'Peak Velocity': max_velocities, 'NVV': nvv, 'NIV': niv, 
        'Mean Acceleration': mean_accels, 'Acceleration Std': std_accels, 'Peak Acceleration': max_accels, 'NIA': nia, 
        'Mean Jerk': mean_jerk, 'Jerk Std': std_jerk, 'Max Jerk': max_jerk, 'NIJ': nij, 
        'Curvature Std': std_curvature, 
        'Duration': duration, 'Total Displacement': total_displacement, 'Number of Strokes': num_of_strokes, 
        'Mean Force': mean_force, 'Force Std': std_force, 'Max Force': max_force
    })

    frame_list.append(df)
    return pd.concat(frame_list,ignore_index=True)